# Queries mais elaboradas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName('SparkEleicoes2022').getOrCreate()

/home/joaopedro/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


# Carregando os dados

'Carregando' o arquivo de bens

In [9]:
bens_df = (
    spark
    .read
    .csv(
        './bem_candidato_2022/bem_candidato_2022_BRASIL.csv',
        header=True,
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
    .withColumn(
        # substituindo , por . para que o Spark entenda que é um float
        # e convertendo para float
        'VR_BEM_CANDIDATO',
        F.regexp_replace('VR_BEM_CANDIDATO', ',', '.').cast('float')
    )
    .select(
        'SQ_CANDIDATO',
        'DS_TIPO_BEM_CANDIDATO',
        'VR_BEM_CANDIDATO'
    )
)

 Carregando o arquivo de candidatos

In [10]:
candidatos_df = (
    spark
    .read
    .csv(
        './consulta_cand_2022/consulta_cand_2022_BRASIL.csv',
        header=True,
        inferSchema=True,
        sep=';',
        encoding='latin1'
    )
)

Realizando o join entre os dois arquivos

In [11]:
bens_candidatos_df = bens_df.join(
    candidatos_df,
    'SQ_CANDIDATO',
    how = 'left'
)

## Valor total de bens declarados

In [23]:
bens_df.agg(
    F.sum('VR_BEM_CANDIDATO').alias('total_bens')
).show( truncate=False) 

+---------------------+
|total_bens           |
+---------------------+
|2.3072723030748222E10|
+---------------------+



## Valor total em bens declarados por UF

In [64]:
(
    bens_candidatos_df
    .groupBy(
        'SG_UF'
    )
    .agg(
        F.sum('VR_BEM_CANDIDATO').alias('total_bens')
    )
    # dividindo por 1.000.000.0000 para converter para bilhões
    # e arrerondando para duas casas decimais
    .withColumn(
        'total_bens',
        F.round( F.col('total_bens') / 1e9, 2 )
    )
    .sort(
        'total_bens',
        ascending=False
    )
    .show(100)
)

+-----+----------+
|SG_UF|total_bens|
+-----+----------+
|   SP|      2.36|
|   GO|      2.36|
|   MG|      2.05|
|   DF|      1.91|
|   MT|      1.18|
|   SC|      1.17|
|   RJ|      1.13|
|   PA|      1.13|
|   PR|      1.08|
|   RS|       1.0|
|   RO|       0.9|
|   BA|      0.73|
|   CE|      0.67|
|   TO|      0.61|
|   MA|      0.52|
|   MS|       0.5|
|   PB|      0.47|
|   PI|      0.42|
|   PE|      0.41|
|   ES|      0.37|
|   RR|      0.33|
|   AP|      0.32|
|   RN|      0.32|
|   AM|       0.3|
|   AC|      0.26|
|   SE|      0.23|
|   BR|      0.18|
|   AL|      0.17|
| null|      null|
+-----+----------+



## Top 5 tipos de bens com maior valor declarado

In [34]:
## Top 5 tipos de bens com maior valor declarado
(
    bens_candidatos_df
    .groupBy(
        'DS_TIPO_BEM_CANDIDATO'
    )
    .agg(
        F.sum('VR_BEM_CANDIDATO').alias('total_bens')
    )
    .withColumn(
        'total_bens',
        F.round( F.col('total_bens') / 1e9, 2 )
    )
    .sort(
        'total_bens',
        ascending=False
    )
    .show(5, truncate=False)
)

+--------------------------------+----------+
|DS_TIPO_BEM_CANDIDATO           |total_bens|
+--------------------------------+----------+
|Quotas ou quinhões de capital   |3.36      |
|Casa                            |2.8       |
|Outras participações societárias|2.32      |
|Apartamento                     |2.28      |
|OUTROS BENS E DIREITOS          |1.84      |
+--------------------------------+----------+
only showing top 5 rows



## Quantidade de candidatos por cargo

In [48]:
## Quantidade de candidatos por cargo

(
    candidatos_df
    .filter(
        'DS_SITUACAO_CANDIDATURA = "APTO"',
    )
    # Desconsiderando vices e suplentes
    .filter(
        "DS_CARGO NOT LIKE 'VICE%'"
    )
    .filter(
        "DS_CARGO NOT LIKE '%SUPLENTE%'"
    )
    .groupBy(
        'DS_CARGO'
    )
    .agg(
        F.countDistinct('SQ_CANDIDATO').alias('total_candidatos')
    )
    .sort(
        'total_candidatos',
        ascending=True
    )
    .show()
)


+------------------+----------------+
|          DS_CARGO|total_candidatos|
+------------------+----------------+
|        PRESIDENTE|              11|
|        GOVERNADOR|             216|
|           SENADOR|             224|
|DEPUTADO DISTRITAL|             586|
|  DEPUTADO FEDERAL|            9823|
| DEPUTADO ESTADUAL|           15742|
+------------------+----------------+



# Top 10 candidatos mais ricos

In [54]:
# Top 10 candidatos mais ricos

(
    bens_candidatos_df
    .filter(
        'DS_SITUACAO_CANDIDATURA = "APTO"',
    )
    .groupBy(
        'SQ_CANDIDATO',
        'NM_CANDIDATO',
        'DS_CARGO',
        'SG_UF',
        'SG_PARTIDO'
    )
    .agg(
        F.sum('VR_BEM_CANDIDATO').alias('Total em Milhões')
    )
    .withColumn(
        'Total em Milhões',
        F.round( F.col('Total em Milhões') / 1e6, 2 )
    )
    .sort(
        'Total em Milhões',
        ascending=False
    )
    .show(10, truncate=False)
)

+------------+---------------------------+-----------------+-----+------------+----------------+
|SQ_CANDIDATO|NM_CANDIDATO               |DS_CARGO         |SG_UF|SG_PARTIDO  |Total em Milhões|
+------------+---------------------------+-----------------+-----+------------+----------------+
|90001615125 |MARCOS ERMÍRIO DE MORAES   |2º SUPLENTE      |GO   |PSDB        |1267.95         |
|70001723172 |PAULO OCTÁVIO ALVES PEREIRA|GOVERNADOR       |DF   |PSD         |618.87          |
|140001611624|AILSON SOUTO DA TRINDADE   |DEPUTADO ESTADUAL|PA   |PP          |448.45          |
|240001605916|ANTIDIO ALEIXO LUNELLI     |DEPUTADO ESTADUAL|SC   |MDB         |390.03          |
|110001643542|OTAVIANO OLAVO PIVETTA     |VICE-GOVERNADOR  |MT   |REPUBLICANOS|378.87          |
|210001609945|ROBERTO ARGENTA            |GOVERNADOR       |RS   |PSC         |372.94          |
|70001621520 |LUIZ OSVALDO PASTORE       |1º SUPLENTE      |DF   |MDB         |367.94          |
|220001614362|JOÃO GONÇALVES F

## Candidato mais rico de cada cargo concorrido

In [59]:
# import window functions 
from pyspark.sql.window import Window

In [62]:
## Candidato Primeiro candidato mais rico por DS_CARGO

(
    bens_candidatos_df
    .filter(
        'DS_SITUACAO_CANDIDATURA = "APTO"',
    )
    .groupBy(
        'SQ_CANDIDATO',
        'NM_CANDIDATO',
        'DS_CARGO',
        'SG_UF',
        'SG_PARTIDO'
    )
    .agg(
        F.sum('VR_BEM_CANDIDATO').alias('Total em Milhões')
    )
    .withColumn(
        'Total em Milhões',
        F.round( F.col('Total em Milhões') / 1e6, 2 )
    )
    .withColumn(
        'rank',
        F.row_number().over(
            Window.partitionBy('DS_CARGO').orderBy(
                F.col('Total em Milhões').desc()
            )
        )
    )
    .filter(
        'rank = 1'
    )
    # drop rank column
    .drop('rank')
    .sort(
        'Total em Milhões',
        ascending=False
    )
).show(10, truncate=False)


+------------+---------------------------+------------------+-----+------------+----------------+
|SQ_CANDIDATO|NM_CANDIDATO               |DS_CARGO          |SG_UF|SG_PARTIDO  |Total em Milhões|
+------------+---------------------------+------------------+-----+------------+----------------+
|90001615125 |MARCOS ERMÍRIO DE MORAES   |2º SUPLENTE       |GO   |PSDB        |1267.95         |
|70001723172 |PAULO OCTÁVIO ALVES PEREIRA|GOVERNADOR        |DF   |PSD         |618.87          |
|140001611624|AILSON SOUTO DA TRINDADE   |DEPUTADO ESTADUAL |PA   |PP          |448.45          |
|110001643542|OTAVIANO OLAVO PIVETTA     |VICE-GOVERNADOR   |MT   |REPUBLICANOS|378.87          |
|70001621520 |LUIZ OSVALDO PASTORE       |1º SUPLENTE       |DF   |MDB         |367.94          |
|60001643110 |EUNÍCIO LOPES DE OLIVEIRA  |DEPUTADO FEDERAL  |CE   |MDB         |158.18          |
|220001713729|JAIME MAXIMINO BAGATTOLI   |SENADOR           |RO   |PL          |55.74           |
|280001603612|LUIZ F

## Total de bens declarados dos candidatos a Deputado Federal por partido

In [63]:
# Total de bens declarados dos candidatos a Deputado Federal por partido

(
    bens_candidatos_df
    .filter(
        'DS_SITUACAO_CANDIDATURA = "APTO"',
    )
    .filter(
        'DS_CARGO = "DEPUTADO FEDERAL"'
    )
    .groupBy(
        'SG_PARTIDO'
    )
    .agg(
        F.sum('VR_BEM_CANDIDATO').alias('Total em Milhões')
    )
    .withColumn(
        'Total em Milhões',
        F.round( F.col('Total em Milhões') / 1e6, 2 )
    )
    .sort(
        'Total em Milhões',
        ascending=False
    )
    .show(100, truncate=False)
)

+-------------+----------------+
|SG_PARTIDO   |Total em Milhões|
+-------------+----------------+
|MDB          |733.37          |
|PP           |604.4           |
|PL           |583.52          |
|UNIÃO        |580.37          |
|PSD          |508.58          |
|REPUBLICANOS |381.56          |
|AVANTE       |374.4           |
|NOVO         |295.83          |
|PTB          |277.08          |
|PSB          |273.78          |
|PDT          |272.64          |
|PODE         |271.34          |
|PSDB         |259.91          |
|PATRIOTA     |226.39          |
|PSC          |215.95          |
|PT           |210.87          |
|PROS         |205.91          |
|PV           |147.67          |
|SOLIDARIEDADE|122.81          |
|AGIR         |119.74          |
|PRTB         |111.92          |
|DC           |102.17          |
|CIDADANIA    |79.53           |
|REDE         |53.98           |
|PMB          |52.0            |
|PMN          |49.17           |
|PSOL         |49.09           |
|PC do B  